In [1]:
import numpy as np
from tensorflow import keras
from motif_search.motif_utilities import plot_weights
path = "/home/steve/Documents/main_storage/resource_data/tep/outputsweights_rnatracker_almost15epochs.h5"

2023-07-11 17:22:47.698609: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-11 17:22:48.100045: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-11 17:22:48.102007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 17:22:49.263076: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load your trained model
#path = '/content/gdrive/MyDrive/Uni/Machine Learning in Regulatory Genomics/scripte/models/'
model = keras.models.load_model(path)  #(path + 'moitfTest3.keras')

model.summary()

conv_layer = model.get_layer('conv1d')  # Replace 'conv1d' with the actual name of your convolutional layer
weights = conv_layer.get_weights()[0]

/home/steve/anaconda3/envs/reggen/lib/python3.11/site-packages/keras/layers/core/lambda_layer.py:325: UserWarning: models.RNAtracker is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


ValueError: bad marshal data (unknown type code)

In [ ]:
print(weights.shape) #channels * input * kernel_size

# tensorflow(kernel_size, input_shape, n_filters)
for i in range(64): #weights.shape[2]
  w = np.array(weights[:,:,i]) #having np.array is important!
  plot_weights(w)